In [1]:
import pandas as pd
import urllib.request
import zipfile

In [2]:
url="https://github.com/mattharrison/datasets/raw/master/data/kaggle-survey-2018.zip"
fname='kaggle-survey-2018.zip'
member_name='multipleChoiceResponses.csv'


In [3]:
def extract_zip(src,dest, member_name):
    url=src
    fname=dest
    fin=urllib.request.urlopen(url)
    data=fin.read()

    with open(dest,mode='wb') as fout:
        fout.write(data)
    with zipfile.ZipFile(dest) as z:
        kag=pd.read_csv(z.open(member_name))
        kag_question=kag.iloc[0]
        raw=kag.iloc[1:]
        return raw

In [4]:
raw = extract_zip(url,fname,member_name)

C:\Users\Tyreek ALEXANDER\AppData\Local\Temp\ipykernel_6964\2146811015.py:10: DtypeWarning: Columns (0,2,8,10,21,23,24,25,26,27,28,44,56,64,83,85,87,107,109,123,125,150,157,172,174,194,210,218,219,223,246,249,262,264,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,304,306,325,326,329,341,368,371,384,385,389,390,391,393,394) have mixed types. Specify dtype option on import or set low_memory=False.
  kag=pd.read_csv(z.open(member_name))


In [6]:
def topn(ser, n=5, default='other'):
    counts = ser.value_counts()
    return ser.where(ser.isin(counts.index[:n]), default)

def tweak_kag(df):
    return (df
        .loc[df.Q3.isin(["United States of America", "China", "India"]) &
             df.Q6.isin(["Data Scientist", "Software Engineer"])]
        .pipe(lambda df_:
            df_.assign(**(df_.Q1.pipe(pd.get_dummies, drop_first=True, prefix='gender')),
                       age=df_.Q2.str.slice(0,2).astype(int),
                       **(df_.Q3.pipe(pd.get_dummies, drop_first=True, prefix='country')),
                       education=df_.Q4.replace({'Master’s degree': 18,
                         'Bachelor’s degree': 16,
                         'Doctoral degree': 20,
                         'Some college/university study without earning a bachelor’s degree': 13,
                         'Professional degree': 19,
                         'I prefer not to answer': None,
                         'No formal education past high school': 12}),
                       **(df_.Q5
                              .pipe(topn, n=3)
                              .replace({
                        'Computer science (software engineering, etc.)': 'cs',
                        'Engineering (non-computer focused)': 'eng',
                        'Mathematics or statistics': 'stat'})
                              .pipe(pd.get_dummies, drop_first=True, prefix='major')),
                       title=df_.Q6,
                       years_exp=(df_.Q8.str.replace('+','', regex=False)
                           .str.split('-', expand=True)
                           .iloc[:,0]
                           .astype(float)),
                       compensation=(df_.Q9.str.replace('+','', regex=False)
                           .str.replace(',','', regex=False)
                           .str.replace('500000', '500', regex=False)
                           .str.replace('I do not wish to disclose my approximate yearly compensation', '0', regex=False)
                           .str.split('-', expand=True)
                           .iloc[:,0]
                           .fillna(0)
                           .astype(int)
                           .mul(1_000)
                                    ),
                       python=df_.Q16_Part_1.fillna(0).replace('Python', 1),
                       r=df_.Q16_Part_2.fillna(0).replace('R', 1),
                       sql=df_.Q16_Part_3.fillna(0).replace('SQL', 1)
               )#assign
              
        )#pipe
        .rename(columns=lambda col:col.replace(' ', '_'))
        .loc[:, 'gender_Male':]   
        .dropna()
       )

In [17]:
def hi(wow):
    return(wow
        #    .pop()
           .append(wow.append([1]))
           )

print(hi(["word"]))


None
